In [1]:
from langchain import PromptTemplate 
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain.document_loaders import PyMuPDFLoader,DirectoryLoader
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [66]:
def load_pdf(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents


In [67]:
extracted_data = load_pdf('data/')

In [65]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=200,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    
    return text_chunks

In [66]:
text_chunks=text_split(extracted_data)
print(len(text_chunks))

7313


In [67]:
model_name = 'sentence-transformers/all-MiniLM-L6-v2'

#download embedding model
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name=model_name) 
    return embeddings


In [68]:
embeddings=download_hugging_face_embeddings()

In [12]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [ ]:
from langchain.vectorstores import FAISS

DB_FAISS_PATH = "vectorstores/db_faiss"
db = FAISS.from_documents(text_chunks, embeddings)
db.save_local(DB_FAISS_PATH)